![Alt text](http://www.ucm.es/logo/ucm.png "a title")

<div align="center"> 
<font size=6> Máster en Nuevas Tecnologías Electrónicas y Fotónicas </font>
</div>
    
<div align="center"> 
<font size=5> Óptica Digital, curso 2021-2022 </font>
</div>

    
<div align="center"> 
<font size=5> Laboratorio 1 - Funcionamiento de un Modulador Espacial de Luz</font>
</div>


- **Grupo de laboratorio**: ***
        
- **Alumnos**:  
     1. Apellidos, nombre
     1. Apellidos, nombre
     1. Apellidos, nombre
     
     
- **Fecha de realización del laboratorio**: aaaa/mm/dd -- hh:mm a hh:mm

# Objetivos de la práctica

    • Aprender a utilizar los equipos del laboratorio.
    • Encontrar el plano del SLM después del sistema 4f.
    • Encontrar las condiciones de modulación de amplitud y fase
    • Implementar una máscara sencilla.


# Introducción
El objetivo de la siguiente práctica es aprender a utilizar todos los aparatos que se emplearán durante las prácticas de Óptica Digital. Esto se lleva a cabo a través de la librería **py_lab**, una librería capaz de controlar diversos motores, cámaras y SLM independientemente del modelo conectado.

Es necesario aprender a realizar las siguientes tareas:
- Colocar el motor que mueve la cámara en la posición deseada.
- Abrir una ventana que muestre lo que ve la cámara en tiempo real.
- Tomar imágenes con las propiedades (resolución, brillo, ...) deseadas.
- Enviar una máscara al SLM.
- Encontrar una posición de los polarizadores y retardadores que implementen modulación de amplitud y otra que implemente modulación de fase.

El resultado archivo jupyter (.ipynb) y su copia como .html es suficiente para realizar la práctica.


---
# Puesta a punto

En primer lugar, hay que cargar los módulos necesarios para el desarrollo de la práctica. Estos módulos son *py_lab*, que servirá para controlar los aparatos, y *diffractio*, con el que implementaremos algunas máscaras para enviarlas al SLM.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from PIL import Image

from py_lab.motor import Motor
from py_lab.camera import Camera
from py_lab.slm import SLM
from py_lab.daca import DACA

from diffractio import degrees, mm, plt, sp, um, np
from diffractio.scalar_masks_XY import Scalar_mask_XY

Después, hay que asegurarse que los datos se guardan en el directorio correcto. Por ello, se debe crear la carpeta dentro de H:\Optica digital\2021_2022\sesion 1\Pareja X. A continuación, se debe establecer ese directorio como el directorio activo:

In [ ]:
#Dependiendo de donde se abra la ventana de comandos, funcionará uno de los dos. Comentad el otro:
%cd H:\Optica digital\2021_2022\sesion 1\Pareja 0
%cd Pareja 0

In [ ]:
%pwd

In [ ]:
import matplotlib

from matplotlib import rcParams
rcParams['figure.dpi']=300
rcParams['image.cmap']='hot'

---
# Control básico de los equipos

Vamos a ver cómo se controla cada uno de los aparatos de la práctica. Iremos de uno en uno para que sea más fácil de aprender. De todas maneras, conviene tener presente que el uso de cualquiera de los aparatos sigue una misma estructura. 

En primer lugar, cada aparato será controlado mediante un objeto de la clase adecuada: Camera para la cámara, Motor para el motor que mueve la cámara, SLM para el SLM, y DACA para la tarjeta de adquisición de datos (DAta Acquisition Card) que mide la señal generada por los fotodetectores. Cualquier operación que se lleve a cabo con dicho aparato se realizará empleando un método de la clase correspondiente.

El uso de cualquiera de estos aparatos segurá la siguiente estructura:
1. Creación del objeto (*cam = Camera()*)
1. Inicialización del objeto (*cam.Open()*). Este paso conecta el ordenador al aparato y comprueba que no haya errores.
1. Uso del dispositivo (*image = cam.Get_Image()*). Cada elemento tiene diversos métodos para realizar las operaciones encesarias.
1. Apagado (*cam.Close()*). Al finalizar el uso hay que terminar la conexión entre el ordenador y el dispositivo.

## Cámara

Vamos a empezar por la cámara. 

### Primeros pasos

Empezamos creando el objeto. Todos los métodos de creación de objeto tienen un argumento de entrada *name* en el que se les especifica el modelo de objeto que se está utilizando. En el caso de la cámara, hay que introducir un nombre diferente dependiendo del puesto de la práctica:

In [ ]:
# Comentar la línea de código correspondiente al otro puesto
cam = Camera(name="ImagingSource")    # Puesto 1
# cam = Camera(name="ImagingSource2")   # Puesto 2

Ahora tenemos que inicializarlo (establecer la conexión entre el ordenador y la cámara). Este paso bloqueará el uso de la cámara mediante otro software.

In [ ]:
cam.Open()
cam.List_Devices()

### Propiedades de la cámara

Ya podemos empezar a usar la cámara. Primero vamos a aprender a cambiar algunas de las propiedades de la cámara. Esto se hace con el método *Set_Property*. Este método acepta 3 variables de entrada:

* **name** (str): Nombre de la propiedad. Posibles opciones: Resolution (resolución), Gain (ganancia), Exposure (tiempo de exposición logaritmico), Framerate (tasa de refresco en Hz), y otras muchas.
* **value**: Nuevo valor de la propiedad.
* **is_switcher** (bool): Determina si la propiedad es de tipo switcher (encendido, apagado) o no. Para la mayoría de propiedades, representa si el valor puede ser variado automáticamente por los drivers de la cámarta o no. Por defecto: False.

<div class="alert alert-block alert-success">
    
<b>Cámara:</b>
La exposición y la ganancia se deben modificar según el experimento para capturar las imágenes con el mejor rango. Hay que procurar que no haya muchos píxeles de la cámara saturados (valor > 255). Los valores adecuados para estos parámetros son:
- Gain: 0 a 5
- Exposure: -5 a -13. Viene dado por el valor 'value=int(np.log2(1/tiempo_integracion))'.
</div>

In [ ]:
cam.Set_Property(name="Resolution", value="Y800 (640x480)", is_switcher=False)   # Cambiar la resolucion a una manejable para el video en vivo
cam.Set_Property(name="Gain", value=0, is_switcher=True)                         # Cambiar la ganancia a manual
cam.Set_Property(name="Exposure", value=0, is_switcher=True)                     # Cambiar el tiempo de exposicion a manual
cam.Set_Property(name="Gain", value=10, is_switcher=False)                       # Cambiar la ganancia a un valor fijo

También podemos preguntar qué valor tienen esas propiedades actualmente con el método *Get_Property*:

In [ ]:
_ = cam.Get_Property(name="Resolution", is_switcher=False)

### Video en vivo

Para que la cámara empieze a medir hay que usar el método *Start_Live*. Si se usa el parámetro de entrada *view* como True, se abrirá una ventana con la imagen en vivo.

In [ ]:
cam.Start_Live(view=True)

<div class="alert alert-block alert-success">
    
<b>Consejo:</b>
Conviene usar la resolución Y800 (640x480) cuando abrimos la ventana para ver la imagen de la cámara para que quepa cómodamente en la pantalla.
    
</div>

Podemos pararlo empleando el método *Stop_Live* (o cerrando la ventana, o presionando Esc con la ventana activa):

In [ ]:
cam.Stop_Live()

### Adquisicion de imagenes

Para tomar imagenes se usa el metodo *Get_Image*. Si además especificamos la variable *draw* como True, podremos ver una representación de la imagen el el notebook.

<div class="alert alert-block alert-info">

**NOTA:** Para tomar imágenes, la cámara debe estar activa (método *Start_Live*).
  
</div>

In [ ]:
cam.Start_Live(view=False)

In [ ]:
image = cam.Get_Image(draw=True)

<div class="alert alert-block alert-success">
    
<b>Consejo:</b>
A la hora de guardar imágenes, conviene usar la máxima resolución Y800 (2592x1944).
    
</div>

### Guardar imágenes

El método *Get_Image* devuelve la imagen en forma de un *numpy.ndarray* de dos dimensiones. Podemos guardar este array empleando el método de numpy *savez*:

In [ ]:
np.savez('Test_image.npz', nombre_variable=image)

Estos archivos .npz se pueden cargar mediante el método de numpy *load*. Todas las variables guardadas en el archivo se cargan como un diccionaro.

In [ ]:
data = np.load('Test_image.npz')
image = data["nombre_variable"]

Otra opción es guardar la información como imágenes. Esto se consigue empleando el módulo *PIL*:

In [ ]:
image_PIL = Image.fromarray(image)
image_PIL.save("Test_image.png")

Estas imágenes pueden cargarse y transformarse a arrays de numpy:

In [ ]:
image_PIL = Image.open('Test_image.png')
image = np.asarray(image_PIL)

### Cerrar la cámara

Por último, vamos a cerrar la cámara.

In [ ]:
cam.Close()

## Motores

Ahora vamos a aprender a controlar los motores.

### Primeros pasos

Como siempre, empezamos creando el objeto e inicializandolo.

In [ ]:
# Comentar la línea de código correspondiente al otro puesto
motor = Motor(name="SMC100")    # Puesto 1
# motor = Motor(name="???")   # Puesto 2

In [ ]:
motor.Open()

In [ ]:
motor.Test_Connection()

Aquellos motores que tienen un encoder que les permite conocer su posición absoluta necesitan encontrar su posición para poder trabajar. Normalmente, esto supone que se mueven a su posición de 0, ubicada en el centro o en uno de los extremos. A esta maniobra se la conoce como *Homing*.

In [ ]:
motor.Home()

<div class="alert alert-block alert-info">

**IMPORTANTE (Puesto 2):** Este motor devuelve el control al programa antes de terminar su movimiento. SIn embargo, cuando se le intenta dar una nueva orden mientras aún se está moviendo, devuelve un error. Hay que esperar a que termine el movimiento antes de dar una nueva orden, o se obtendrá un error. 
  
</div>

### Movimiento

Tenemos varias opciones para movernos. En primer lugar, está el método *Move_Absolute*. Con este método podemos movernos a una posición específica.

In [ ]:
motor.Move_Absolute(pos=25, units="mm")

La segunda opción es el movimiento relativo, que nos permite movernos una cierta distancia respecto a la posición actual. Para ello se usa el método *Move_Relative*. El signo de la distancia define la dirección de movimiento.

In [ ]:
motor.Move_Relative(dist=-30, units="mm")

<div class="alert alert-block alert-success">
    
<b>Nota:</b>
Los métodos de movimiento ofrecen como argumento de salida la posición final.
    
</div>

Para conocer la posición actual se emplea el método *Get_Position*:

In [ ]:
pos = motor.Get_Position()
print(pos)

### Guardar posiciones

Es posible guardar la posición actual para poder volver a ella más tarde fácilemnte.

In [ ]:
motor.Save_Position(name="Inicio")

Después, se puede volver a dicha posición empleando el método *Move_To_Position*:

In [ ]:
motor.Move_Relative(dist=10)

In [ ]:
motor.Move_To_Position(name="Inicio")

### Cerrar el motor

Como siempre, hay que cerrar el objeto después de terminar de utilizarlo.

In [ ]:
motor.Close()

## SLM

Ahora vamos a manejar el SLM.

### Primeros pasos

Como siempre, empezamos creando el objeto. A diferencia de otros elementos, no es necesario ejecutar ningún método de conexión, ya que los SLM funcionan de una manera muy peculiar: el ordenador los trata como una pantalla.

In [ ]:
# Comentar la línea de código correspondiente al otro puesto
slm = SLM(name="HoloEye2500")    # Puesto 1
# slm = SLM(name="HoloEyePluto")   # Puesto 2

<div class="alert alert-block alert-success">
    
<b>Nota:</b>
Se puede comprobar que el SLM es tratado como una segunda pantalla haciendo una captura de pantalla y pegandola en Paint.
    
</div>

### Envío de máscaras

La manera de usar el SLM es enviarle una máscara para que él la represente en sus píxeles. Esto se consigue mediante el método *Send_Image*. La variable de entrada principal del método es *image*, que representa qué va a mandarse al SLM. Esta variable puede ser un numpy.ndarray o un diffractio.Scalar_mask_XY. Vamos a empezar con el array de numpy.

In [ ]:
image = np.random.rand(768, 1024)    # Puesto 1
# image = np.random.rand(1080, 1920)   # Puesto 2
slm.Send_Image(image)

<div class="alert alert-block alert-info">

**NOTA:** La resolución de la imagen debe coincidir con la del SLM para que se visualice correctamente.
  
</div>

Usar una máscara de diffractio es un poco más complicado, ya que es necesario definir las coordenadas espaciales. Estas coordenadas dependen del SLM que se esté utilizando. Para hacerlo un poco más fácil, vamos a usar la siguiente función:

In [ ]:
def Get_Diffractio_Space(slm):
    """Calculates diffractio x and y variables for diffractio.

    Returns:
        x, y (np.ndarray): Arrays of space.
    """
    lim = slm.pixel_size[0] * mm * (slm.resolution[0] - 1) / 2
    x = np.linspace(-lim, lim, slm.resolution[0])
    lim = slm.pixel_size[1] * mm * (slm.resolution[1] - 1) / 2
    y = np.linspace(-lim, lim, slm.resolution[1])
    return x, y

In [ ]:
# Calcular el espacio
x, y = Get_Diffractio_Space(slm)

# Calcular la máscara
ajedrez = Scalar_mask_XY(x=x, y=y, wavelength=0.6328*um)
ajedrez.grating_2D_chess(period=2 * mm, amin=0, amax=1., phase=0,
                r0=(0, 0), fill_factor=0.5, angle=0.0 * degrees)
ajedrez.draw(kind='intensity', has_colorbar='horizontal')

# Mandarla al SLM
slm.Send_Image(ajedrez, kind="intensity")

### Cerrar el SLM

El SLM, puesto que no se inicializa, no necesita cerrarse. Sin embargo, es posible que sea necesario cerrar la ventana que muestra la imagen del SLM en la pantaya auxiliar. Para eso, se emplea el método *Close_Image*.

In [ ]:
slm.Close_Image()

# Ejercicios

Debéis realizar los siguientes ejercicios.

## Encontrar el plano del SLM

Puesto que hay un sistema 4f después del SLM, el plano del SLM se reproduce en el plano focal imagen de la segunda lente. Para encontrarlo con la cámara hay que realizar las siguientes operaciones:

1. Colocar una imagen en el SLM que presente alto contraste (por ejemplo, la imagen del ajedrez).
1. Abrir una ventana que muestre lo que mide la cámara.
1. Mover el motor hasta ver bien enfocado la imagen proyectada en el SLM.
1. Guardar la imagen.

No olvidéis anotar la posición absoluta de este plano, la vais a utilizar en otras prácticas.

## Encontrar las configuraciones de modulación de amplitud y fase

Para encontrarlas tendréis que rotar los polarizadores y retardadores del PSG y el PSA. El proceso es el siguiente:

1. Colocar la cámara en el plano del SLM.
1. Proyectar una imagen en el SLM que contenga regiones de nivel 0 y regiones de nivel 1 (por ejemplo, la de ajedrez).
1. Rotar los elementos del PSG y el PSA hasta que se vea una imagen con cuadrados muy negros y muy blancos (amplitud), o cuadrados con todos el mismo brillo (fase).
1. Guardar una imagen para cada configuración.
1. Calcular (ya en casa) el nivel de gris medio medido por la cámara, y su error, de los cuadrados negros y los cuadrados blancos. En el caso de la modulación de amplitud, calcular la visibilidad.

Tenéis que anotar los ángulos a los que habéis colocado los elementos de polarización para poder recuperas esas configuraciones en prácticas subsiguientes.

<div class="alert alert-block alert-info">

**IMPORTANTE:** No olvides guardar este archivo, una vez finalizada la práctica, en formatos .ipynb y .html, y guárdalos en la carpeta de tu pareja.
  
</div>

<div class="alert alert-block alert-success">
    
<b>Nota:</b>
Guardaros también los archivos que habéis generado en un pendrive. Recuerda utilizar este archivo, para obtener información para las próximas prácticas.
    
</div>